# Housing Affordability Analysis in Massachusetts

This notebook presents an analysis of the affordability of different types of housing in Massachusetts, with a particular focus on 3-4 unit apartments. The goal is to develop a model that can predict three different cost measures:

1. Monthly Owner Costs With Mortgage
2. Monthly Owner Costs Without Mortgage
3. Monthly Renter Costs

These cost measures are treated as a percentage of income spent on housing costs.

The analysis involves the following key steps:

## 1. Data Loading and Exploration

We begin by loading the dataset and exploring its basic characteristics. This involves understanding the structure of the dataset, the features available, and the nature of the target variables.

## 2. Data Splitting

The dataset is split into a training set and a testing set. The training set is used to train our machine learning models, and the testing set is used to evaluate the models' performance.

## 3. Model Building and Evaluation

We build multiple regression models, including Linear Regression, Random Forest Regression, and Support Vector Regression, for each target variable. Each model is evaluated using Mean Absolute Error (MAE), Mean Squared Error (MSE), and R-squared.

## 4. Hyperparameter Tuning

The models' performance is then improved by tuning their hyperparameters. We use GridSearchCV, which performs an exhaustive search over specified parameter values for an estimator.

In [4]:
# Load the data
df = pd.read_csv("preprocessed_dataframe.csv")
df.head()

,Mobile Home or Trailer,Mobile Home or Trailer.1,Mobile Home or Trailer.2,Mobile Home or Trailer.3,Mobile Home or Trailer.4,Mobile Home or Trailer.5,Mobile Home or Trailer.6,Mobile Home or Trailer.7,One-family house detached,One-family house detached.1,...,"MA_Plymouth County (East)--Plymouth, Marshfield, Scituate, Duxbury & Kingston Towns PUMA; Massachusetts","MA_Suffolk County (North)--Revere, Chelsea & Winthrop Town Cities PUMA; Massachusetts","MA_Weymouth Town, Braintree Town Cities, Hingham, Hull & Cohasset Towns PUMA; Massachusetts","MA_Woburn, Melrose Cities, Saugus, Wakefield & Stoneham Towns PUMA; Massachusetts","MA_Worcester & Middlesex Counties (Outside Leominster, Fitchburg & Gardner Cities) PUMA; Massachusetts","MA_Worcester County (Central)--Worcester City PUMA, Massachusetts","MA_Worcester County (East Central) PUMA, Massachusetts","MA_Worcester County (Northeast)--Leominster, Fitchburg & Gardner Cities PUMA; Massachusetts","MA_Worcester County (South) PUMA, Massachusetts","MA_Worcester County (West Central) PUMA, Massachusetts"
0,0.584913,-0.203924,0.557950,-0.557950,0.456138,0.146246,1.372609,2.970455,0.847298,0.221807,...,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028
1,1.097951,1.425702,-0.004151,0.004151,1.352350,-0.148133,0.586100,-0.949206,0.008192,2.271914,...,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028
2,-0.721656,0.262776,-2.425760,2.425760,-0.559708,-0.172664,-0.298722,-0.223343,-0.320500,1.168822,...,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,7.141428,-0.140028,-0.140028,-0.140028,-0.140028
3,0.007146,-0.601767,0.842268,-0.842268,-0.158827,-0.393448,2.945627,-0.223343,0.138691,0.011945,...,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,7.141428,-0.140028,-0.140028
4,0.604092,-0.601767,0.842268,-0.842268,0.363788,0.538750,-0.397036,-0.223343,0.752998,0.300901,...,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,-0.140028,7.141428


In [5]:
# Splitting the dataset into training and testing sets
X = df.drop(columns=['3-4 Apartments.5', '3-4 Apartments.6', '3-4 Apartments.7'])
y1 = df['3-4 Apartments.5']  # Monthly Owner Costs With Mortgage
y2 = df['3-4 Apartments.6']  # Monthly Owner Costs Without Mortgage
y3 = df['3-4 Apartments.7']  # Monthly Renter Costs

# Splitting into training and test sets for each target variable
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
_, _, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)
_, _, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y1_train.shape, y1_test.shape, y2_train.shape, y2_test.shape, y3_train.shape, y3_test.shape


((41, 176), (11, 176), (41,), (11,), (41,), (11,), (41,), (11,))

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score



# Define function to evaluate models
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return mae, mse, r2

# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Support Vector Regression": SVR()
}

# Define target variables
target_variables = {
    "Monthly Owner Costs With Mortgage": (y1_train, y1_test),
    "Monthly Owner Costs Without Mortgage": (y2_train, y2_test),
    "Monthly Renter Costs": (y3_train, y3_test)
}

# Fit and evaluate models
evaluation_results = []
for target, (y_train, y_test) in target_variables.items():
    for model_name, model in models.items():
        # Fit the model
        model.fit(X_train, y_train)
        
        # Evaluate the model
        mae, mse, r2 = evaluate_model(model, X_test, y_test)
        
        # Store the results
        evaluation_results.append({
            "Target Variable": target,
            "Model": model_name,
            "Mean Absolute Error": mae,
            "Mean Squared Error": mse,
            "R-squared": r2
        })

# Convert results to DataFrame
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df


,Target Variable,Model,Mean Absolute Error,Mean Squared Error,R-squared
0,Monthly Owner Costs With Mortgage,Linear Regression,0.782241,0.891489,-0.263892
1,Monthly Owner Costs With Mortgage,Random Forest,0.798381,0.955565,-0.354735
2,Monthly Owner Costs With Mortgage,Support Vector Regression,0.627879,0.645706,0.084563
3,Monthly Owner Costs Without Mortgage,Linear Regression,1.005252,1.303217,0.031646
4,Monthly Owner Costs Without Mortgage,Random Forest,1.139001,1.718193,-0.276701
5,Monthly Owner Costs Without Mortgage,Support Vector Regression,1.008830,1.405587,-0.044420
6,Monthly Renter Costs,Linear Regression,0.876781,1.081113,-0.172167
7,Monthly Renter Costs,Random Forest,0.841098,1.051498,-0.140058
8,Monthly Renter Costs,Support Vector Regression,0.792248,0.952020,-0.032202


For predicting 'Monthly Owner Costs With Mortgage', the Support Vector Regression has the lowest MAE and MSE and the highest R-squared value, making it the best model for this particular target variable among the ones tested.

For predicting 'Monthly Owner Costs Without Mortgage', the Linear Regression has the lowest MAE and MSE, although the R-squared value is close to zero, which indicates that the model doesn't explain much of the variability in the data.

For predicting 'Monthly Renter Costs', the Support Vector Regression has the lowest MAE and MSE, and the highest R-squared value, making it the best model for this target variable as well among the ones tested.

These results should be taken cautiously since the dataset is quite small and the models may not be very reliable. Furthermore, the negative R-squared values indicate that the models are not fitting the data well. Additional data, feature engineering, and hyperparameter tuning may be needed to improve the performance of the models.

Based on the evaluation results, Support Vector Regression (SVR) seems to be the most promising model for both 'Monthly Owner Costs With Mortgage' and 'Monthly Renter Costs'. For 'Monthly Owner Costs Without Mortgage', although Linear Regression has the lowest errors, its R-squared value is near zero, indicating that the model doesn't explain much of the variability in the data. Therefore, we will also use SVR for this target variable due to its relative performance.

Next, I will perform hyperparameter tuning on the Support Vector Regression model for each of the target variables to see if we can improve its performance. Hyperparameters are parameters that are not learned from the data, but are set prior to the commencement of the learning process. Tuning them properly can lead to better model performance.

I'll use GridSearchCV, which is a method used to perform hyperparameter tuning. It works by training our model multiple times on a range of parameters that we specify. That way, we can check which of our parameters work best on the given model.

For the SVR model, I'll tune the 'C' (Regularization parameter), 'kernel' (Specifies the kernel type to be used in the algorithm), and 'gamma' (Kernel coefficient for 'rbf', 'poly' and 'sigmoid') hyperparameters.

In [12]:
from sklearn.model_selection import GridSearchCV


# Define the hyperparameters for the SVR model
hyperparameters = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Instantiate the GridSearchCV object and fit it to the training data
grid_search_results = []
for target, (y_train, y_test) in target_variables.items():
    grid_search = GridSearchCV(SVR(), hyperparameters, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)
    
    # Evaluate the model
    mae, mse, r2 = evaluate_model(grid_search.best_estimator_, X_test, y_test)
    
    # Store the results
    grid_search_results.append({
        "Target Variable": target,
        "Best Parameters": grid_search.best_params_,
        "Mean Absolute Error": mae,
        "Mean Squared Error": mse,
        "R-squared": r2
    })

# Convert results to DataFrame
grid_search_df = pd.DataFrame(grid_search_results)
grid_search_df


,Target Variable,Best Parameters,Mean Absolute Error,Mean Squared Error,R-squared
0,Monthly Owner Costs With Mortgage,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.670828,0.700477,0.006911
1,Monthly Owner Costs Without Mortgage,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.954889,1.368684,-0.016999
2,Monthly Renter Costs,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'poly'}",0.779873,0.937974,-0.016972


The hyperparameter tuning slightly improved the performance of the SVR model for 'Monthly Owner Costs With Mortgage' and 'Monthly Renter Costs', as indicated by the decrease in MAE and MSE, and the increase in R-squared. However, for 'Monthly Owner Costs Without Mortgage', the errors increased slightly, and the R-squared value remained near zero.

Again, these results should be taken cautiously due to the small dataset and the negative or near zero R-squared values. Additional data, feature engineering, and possibly trying out other machine learning models or techniques may be needed to improve the performance of the models.

For now, the best models for predicting 'Monthly Owner Costs With Mortgage', 'Monthly Owner Costs Without Mortgage', and 'Monthly Renter Costs' are all Support Vector Regressions with their respective best parameters.

In [ ]:
from joblib import dump

# Best SVR model for 'Monthly Owner Costs With Mortgage'
best_model_mortgage = SVR(C=10, gamma='scale', kernel='rbf')
best_model_mortgage.fit(X, y1)
dump(best_model_mortgage, '/mnt/data/best_model_mortgage.joblib')

# Best SVR model for 'Monthly Owner Costs Without Mortgage'
best_model_without_mortgage = SVR(C=10, gamma='auto', kernel='poly')
best_model_without_mortgage.fit(X, y2)
dump(best_model_without_mortgage, '/mnt/data/best_model_without_mortgage.joblib')

# Best SVR model for 'Monthly Renter Costs'
best_model_renter = SVR(C=0.1, gamma='auto', kernel='poly')
best_model_renter.fit(X, y3)
dump(best_model_renter, '/mnt/data/best_model_renter.joblib')

["Models saved successfully"]